# Case Study - The Current

* The Current is an alternative radio station
* We will pull information about the play list.

## <font color="red">Exercise - Go the the following page and inspect the following </font>

* Song title
* Artist
* Play time
* Day, date, period (am/pm)

http://www.thecurrent.org/playlist/2014-01-01/01

In [1]:
import requests
import bs4
import datetime

In [2]:
example_url = 'http://www.thecurrent.org/playlist/2014-01-01/01'
s = requests.Session()
r = s.get(example_url)

soup = bs4.BeautifulSoup(r.content, "lxml")

# Pull off the period of the day (am/pm)

Pull out the "am"/"pm"

1. Inspect the element
2. Identify the html tag and class
3. Search the soup
    1. There should be one item returned
4. Use soup\string methods to pull out the info

In [3]:
# I should have used find here...Why?
soup('span', class_="hour-header open")

[<span class="hour-header open">
      1:00 am to  2:00 am
   </span>]

In [4]:
len(soup('span', class_="hour-header open"))

1

In [5]:
soup('span', class_="hour-header open")[0]

<span class="hour-header open">
     1:00 am to  2:00 am
  </span>

In [6]:
soup('span', class_="hour-header open")[0].next

'\n     1:00 am to  2:00 am\n  '

In [7]:
soup('span', class_="hour-header open")[0].next.split('to')

['\n     1:00 am ', '  2:00 am\n  ']

In [8]:

soup('span', class_="hour-header open")[0].next.split('to')[0]

'\n     1:00 am '

In [9]:
soup('span', class_="hour-header open")[0].next.split('to')[0].rstrip()

'\n     1:00 am'

In [10]:
period = soup('span', class_="hour-header open")[0].next.split('to')[0].rstrip()[-2:]
period

'am'

## <font color="red"> Breakout activity </font>

* Pull out the day of the week

# Title of each song

1. Inspect the element
2. Identify the html tag and class
3. Use `soup.findAll` to make a list of all relevant tags
4. Pull off an example case
5. Use soup/string methods to pull out the title
6. Use a list comprehension to process all tags

In [11]:
soup.findAll('h5', class_ = "title")

[<h5 class="title">Holy Roller</h5>,
 <h5 class="title">Kingdom of Rust</h5>,
 <h5 class="title">Black Dog</h5>,
 <h5 class="title">Turn It Around</h5>,
 <h5 class="title">Flavor of the Month</h5>,
 <h5 class="title">Potential Wife</h5>,
 <h5 class="title">24 Hours</h5>,
 <h5 class="title">Who's Gonna Shoe Your Pretty Little Feet?</h5>,
 <h5 class="title">Marigold</h5>,
 <h5 class="title">High Road</h5>,
 <h5 class="title">The Vampyre Of Time and Memory</h5>,
 <h5 class="title">Valerie Plame</h5>,
 <h5 class="title">Morning Song</h5>,
 <h5 class="title">(You Will) Set The World On Fire</h5>,
 <h5 class="title">Sixteen Saltines</h5>,
 <h5 class="title">Wave of Mutilation</h5>]

In [12]:
example_tag =  soup.findAll('h5', class_ = "title")[0]
example_tag

<h5 class="title">Holy Roller</h5>

In [13]:
example_tag.next

'Holy Roller'

In [14]:
example_tag.next.strip()

'Holy Roller'

In [15]:
titles = [tag.next.strip() for tag in soup.findAll('h5', class_ = "title")]
titles

['Holy Roller',
 'Kingdom of Rust',
 'Black Dog',
 'Turn It Around',
 'Flavor of the Month',
 'Potential Wife',
 '24 Hours',
 "Who's Gonna Shoe Your Pretty Little Feet?",
 'Marigold',
 'High Road',
 'The Vampyre Of Time and Memory',
 'Valerie Plame',
 'Morning Song',
 '(You Will) Set The World On Fire',
 'Sixteen Saltines',
 'Wave of Mutilation']

## <font color="red"> Break out session - Name of the Artist </font>

1. Inspect the element
2. Identify the html tag and class
3. Use `soup.findAll` to make a list of all relevant tags
4. Pull off an example case
5. Use soup/string methods to pull out the title
6. Use a list comprehension to process all tags

## <font color="red"> Break out session - song start time </font>

1. Inspect the element
    1. This one is tricky
    2. Time tag does not have a tag, but
    3. The surrounding div does have a class
2. Identify the html tag and class
3. Use `soup.findAll` to make a list of all relevant tags
4. Pull off an example case
5. Use soup/string methods to pull out the title
6. Use a list comprehension to process all tags

# Putting it all together

In [16]:
def get_period(soup):
    search = soup('span', class_="hour-header open")
    if len(search) > 0:
        return search[0].next.split('to')[0].rstrip()[-2:]
    else:
        return None

def get_day(soup):
    search = soup('a', class_="start-picker")
    if len(search) > 0:
        return search[0].next.split(',')[0]
    else:
        return None
    
def get_song_info(url):
    print("Starting {0} urls".format(url))
    date = url.split('/')[-2]
    s = requests.Session()
    r = s.get(url)
    soup = bs4.BeautifulSoup(r.content, 'lxml')
    period = get_period(soup)
    day_of_week = get_day(soup)
    soup = bs4.BeautifulSoup(r.content)
    titles = [t.next.strip() for t in soup.findAll('h5', class_="title")]
    artists = [a.next.strip() for a in soup.findAll('h5',class_='artist')]
    times = [d.time.next.strip() for d in soup('div', class_="two columns songTime")]
    song_info = [(day_of_week, date, time, period, title, artist) 
             for time, title, artist in zip(times, titles, artists)]
    return song_info

In [17]:
get_song_info(example_url)

Starting http://www.thecurrent.org/playlist/2014-01-01/01 urls


[('Wednesday',
  '2014-01-01',
  '1:59',
  'am',
  'Holy Roller',
  'Thao and The Get Down Stay Down'),
 ('Wednesday', '2014-01-01', '1:54', 'am', 'Kingdom of Rust', 'Doves'),
 ('Wednesday', '2014-01-01', '1:51', 'am', 'Black Dog', 'Frankie Lee'),
 ('Wednesday', '2014-01-01', '1:46', 'am', 'Turn It Around', 'Lucius'),
 ('Wednesday',
  '2014-01-01',
  '1:44',
  'am',
  'Flavor of the Month',
  'The Posies'),
 ('Wednesday', '2014-01-01', '1:38', 'am', 'Potential Wife', 'Strange Names'),
 ('Wednesday', '2014-01-01', '1:34', 'am', '24 Hours', 'Sky Ferreira'),
 ('Wednesday',
  '2014-01-01',
  '1:31',
  'am',
  "Who's Gonna Shoe Your Pretty Little Feet?",
  'Billie Joe and Norah'),
 ('Wednesday',
  '2014-01-01',
  '1:27',
  'am',
  'Marigold',
  'J. Roddy Walston and The Business'),
 ('Wednesday', '2014-01-01', '1:23', 'am', 'High Road', 'Cults'),
 ('Wednesday',
  '2014-01-01',
  '1:19',
  'am',
  'The Vampyre Of Time and Memory',
  'Queens of the Stone Age'),
 ('Wednesday',
  '2014-01-01',


# Collecting a years worth of data

## Step 1 - Identify the url pattern

The Current uses urls of the following pattern

    'http://www.thecurrent.org/playlist/2017-05-04/10'

or 

    'http://www.thecurrent.org/playlist/year-month-day/hour'

## Question: How might you generate all combinations for a given year?

**Answer:** Python has a tool for that!

In [18]:
numdays = 365
base = datetime.datetime.today()
dts = [base - datetime.timedelta(hours = h) for h in range(0, numdays*24)]

In [19]:
def output_address(dt):
    fmt = 'http://www.thecurrent.org/playlist/%Y-%m-%d/%H'
    return dt.strftime(fmt)

def test_output_address():
    date = datetime.datetime(2000,1,1,1)
    assert output_address(date) == 'http://www.thecurrent.org/playlist/2000-01-01/01'
test_output_address()

In [20]:
urls = [output_address(d) for d in dts]
urls[:10]

['http://www.thecurrent.org/playlist/2019-05-13/09',
 'http://www.thecurrent.org/playlist/2019-05-13/08',
 'http://www.thecurrent.org/playlist/2019-05-13/07',
 'http://www.thecurrent.org/playlist/2019-05-13/06',
 'http://www.thecurrent.org/playlist/2019-05-13/05',
 'http://www.thecurrent.org/playlist/2019-05-13/04',
 'http://www.thecurrent.org/playlist/2019-05-13/03',
 'http://www.thecurrent.org/playlist/2019-05-13/02',
 'http://www.thecurrent.org/playlist/2019-05-13/01',
 'http://www.thecurrent.org/playlist/2019-05-13/00']

In [ ]:
from toolz import pipe
from toolz.curried import take
from more_itertools import side_effect, consume


with open('the_current_last_year_new.csv', 'w') as outfile:
    header = "Weekday,Date,Time,Period,Song_Title,Artist"
    print(header, file=outfile)
    count = 0
    for url in urls:
        for song_info in get_song_info(url):
            count += 1
            if count % 5 == 0:
                print("Processed {0} songs".format(count))
            join_info = ",".join(song_info)
            print(join_info, file=outfile)
            if count % 1000 == 0:
                print("Flushing output")
                outfile.flush()